# Objective

In my baseline validation set consist on the last 28 days of the training data. 

The problem in doing this is the following:

Are we sure that this validation aligns with the test set?? (unknown demand and behaviour)

We really don't know, thats what we need to predict xD.

For this reason the best solution is to make a model that can generalize to unseen data (unknown demand and behaviour). Here we are going to validate the entire training data with GroupKFold strategy to avoid data leakage.

The main point on doing this is that the mean of different validations can generalize to a wide range of cases, we will not have the best model for our test but we reduce the chance of overfitting and have a horrible score.

Best model is to get a validation that is really similar to the test set, but we don't know which one it is, this is unknown, we can make some guesses or found a statistical ground to choose the right validation set.

Another problem that we face is the loss function. Root mean squared error is not align with our competition metric (lower rmse does not guarantee a smaller wrmsse). For this we will use an asymmetric mean squared error loss function. 

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from typing import Union
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
import gc
import lightgbm as lgb
from datetime import datetime, timedelta
from sklearn.model_selection import GroupKFold
from sklearn import metrics

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
# helper functions to reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
!pwd

/home/jupyter/m5-forecasting-kaggle/Code


In [8]:
# function to read our data
def read_data():
    # data_small.pkl has been prepared using the all the raw competition data and data_small.pkl is provided as input data for this notebook.
    # read data
#     data = pd.read_pickle('/kaggle/input/m5-reduce-data/data_small.pkl')
    data = pd.read_pickle('../Data/data_small.pkl')
    # fillna and label encode categorical features
    data = transform(data)
    # read submission
    submission = pd.read_csv('../Data/raw/sample_submission.csv')
    return data, submission

In [9]:
# filla na and label encode categorical features
def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    #fill all NaNs in event related columns with unknown and this unknown will be treated as a level 
    #when these columns are provided as cat vars in lightgbm.
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
           'event_name_2', 'event_type_2']
    # We are label encoding the categorical variables. These label enocded variables will go as categorical varibales in lightgbm.
    # We can also use other categorical encoding such Target, Kfold target, LOO target, Catboost encoding etc. which might improve results.
    # We are using label encoding for the sake of simplicity
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
        
    # reduce memory usage
    data = reduce_mem_usage(data)
    
    return data

Following are some points which are worth taking note of w.r.t feature engineering and validation strategy:

* Actually here for validation I am using __GroupKFold CV__ and it does have future data in it. But it all depends on how you build your features. If i used lag(t-1) or a few other small lags, the leakage would be huge but I am only using rolling features (there would be leakage but it´s hard for model to find a leak when I am using __GroupKFold CV__ with no lag features).

* Leakage in time series is introduced if future data (usually test data) is used for training and then predicting on history (usually train data). When we do normal KFold CV there is a huge chance of leakage since randomization/shuffling will cause training on future and then predicting on history. But here we are using GroupKFold CV (year, and week group), which will make leakage less prominent to a certain extent.

* Leakage depends on the features you are using + cv validation strategy. If you check i made a groupkfold strategy for each week, year. In other words there will be some week missing in training.

* For example if you use lag(t-1), there is a really big chance of leakage. But if you check my features there are all rolling means, std for _demand_ variable.

* On the other hand you can validate with future, past or in between data. Most time series guide recommend __future validation__ or __time series split validation__ or __Walk forward validation__ but in my experience and other competition groupkfold using time batches work as well. Im not saying my model does not have leakeage, but it has somewhat less leakage because of features I am using and validation strategy (__GroupKFold CV__).

* The advantage of doing GroupKfold against time series split is that you validate the entire training data (just like vanilla KFold) but downside is there is still some chance of leakage since we might be training on future data and predicting on history while training and validation.


In [10]:
# simple feature ingineer function
def simple_fe(data):
    
    DAYS_PRED = 28 #We have to forecast for 28 days out. that's why last 28 days of data has to be removed/shifted out.
    periods = [7, 15, 30, 90] #rolling mean/std windows

    # Demand features

    ## Lag features. These lag features will introduce huge leakage with GroupKFold CV. That's why we are avoiding it.
    
    # for diff in [0, 1, 2]:
    #         shift = DAYS_PRED + diff
    #         data[f"shift_t{shift}"] = data.groupby('id')['demand'].transform(lambda x: x.shift(shift))

    ## Rolling mean features
    ## before taking the rolling mean/std we have to shift by DAYS_PRED so that we have these features available for test data too.
    for period in periods:
        data['demand_rolling_mean_t' + str(period)] = data.groupby('id')['demand'].transform(lambda x: x.shift(DAYS_PRED).rolling(period).mean())
        data['demand_rolling_std_t' + str(period)] = data.groupby('id')['demand'].transform(lambda x: x.shift(DAYS_PRED).rolling(period).std())

    # Price features

    data["shift_price_t1"] = data.groupby(["id"])["sell_price"].transform(lambda x: x.shift(1))
    data["price_change_t1"] = (data["shift_price_t1"] - data["sell_price"]) / (data["shift_price_t1"]) #rate of price change day by day
    data["rolling_price_max_t365"] = data.groupby('id')["sell_price"].transform(lambda x: x.shift(1).rolling(365).max())
    data["price_change_t365"] = (data["rolling_price_max_t365"] - data["sell_price"]) / (data["rolling_price_max_t365"]) #rate of price change over a year
    data["rolling_price_std_t7"] = data.groupby('id')["sell_price"].transform(lambda x: x.rolling(7).std())
    data["rolling_price_std_t30"] = data.groupby('id')["sell_price"].transform(lambda x: x.rolling(30).std())


    # reduce memory
    data = reduce_mem_usage(data)
    # get time features
    data['date'] = pd.to_datetime(data['date'])
    time_features = ["year", "quarter", "month", "week", "day","dayofweek", 'dayofyear', "is_year_end", "is_year_start", 
                     "is_quarter_end", "is_quarter_start", "is_month_end", "is_month_start"]
#     time_features = ["year", "quarter", "month", "week", "day","dayofweek", 'dayofyear']
    dtype = np.int16
    for time_feature in time_features:
        data[time_feature] = getattr(data['date'].dt, time_feature).astype(dtype)
    
    data["is_weekend"] = data["dayofweek"].isin([5, 6]).astype(dtype)

    return data

In [11]:
# self-defined/custom objective function. This is how custom objective/loss function is used in LighGBM and similar models like XgBoost and Catboost
# f(y_preds: array, train_data: Dataset) -> grad: array, hess: array  -- With Lightgbm API
# f(y_preds: array, y_true: array) -> grad: array, hess: array  -- With Sklearn API
# read more about different custom loss function for Xgboost/lightgbm here https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564

def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
    hess = np.where(residual < 0, 2, 2 * 1.15)
    return grad, hess

In [12]:
# self-defined/custom eval metric
# f(y_preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool -- With Lightgbm API
# f(y_preds: array, y_true: array) -> name: string, eval_result: float, is_higher_better: bool -- With Sklearn API

# NOTE: when you do customized loss function, the default prediction value is margin
# This may make built-in evalution metric calculate wrong results
# For example, we are doing log likelihood loss, the prediction is score before logistic transformation
# Keep this in mind when you use the customization

def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * 1.15) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [13]:
# define lgbm simple model
def run_lgb(data, features, cat_features):
    
    # reset_index
    data.reset_index(inplace = True, drop = True)
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-04-24'] #end date in the data is 2016-05-22. Thats's why leaving last 28 days (before '2016-04-25') is train.
    y_train = x_train['demand']
    test = data[data['date'] >= '2016-04-25'] #end date in the data is 2016-05-22. Thats's why last 28 days (starting from  '2016-04-25') is test.

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.1,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        'min_data_per_leaf': 25,
        'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5}
    
    oof = np.zeros(len(x_train))
    preds = np.zeros(len(test))
    
    # GroupKFold by week, month to avoid leakage and overfitting (not entirely sure xD)
    kf = GroupKFold(5)
    # get subgroups for each week, year pair
    group = x_train['week'].astype(str) + '_' + x_train['year'].astype(str)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train, group)):
        print(f'Training fold {fold + 1}')
        train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], 
                                categorical_feature = cat_features)
        val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], 
                              categorical_feature = cat_features)
        
        # train with our custom loss function and evaluation metric 
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                          valid_sets = [train_set, val_set], verbose_eval = 50, fobj = custom_asymmetric_train, 
                          feval = custom_asymmetric_valid)
    
        # predict oof
        oof[val_idx] = model.predict(x_train.iloc[val_idx][features])

        # predict test
        preds += model.predict(test[features]) / 5
        
        print('-'*50)
        print('\n')
        
    oof_rmse = np.sqrt(metrics.mean_squared_error(y_train, oof))
    print(f'Our out of folds rmse is {oof_rmse}')
        
    test = test[['id', 'date', 'demand']]
    test['demand'] = preds
    return test


In [14]:
# function to get the predictions in the correct format
def predict(test, submission):
    predictions = pd.pivot(test, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission_custom_loss.csv', index = False)

In [15]:
train_eval= pd.read_csv("../Data/raw/sales_train_evaluation.csv")
train_valid= pd.read_csv("../Data/raw/sales_train_validation.csv")
submission= pd.read_csv("../Data/raw/sample_submission.csv")
calendar= pd.read_csv("../Data/raw/calendar.csv")
sell_price= pd.read_csv("../Data/raw/sell_prices.csv")
train_eval.shape, train_valid.shape, submission.shape, calendar.shape, sell_price.shape

((30490, 1947), (30490, 1919), (60980, 29), (1969, 14), (6841121, 4))

In [16]:
# print(train_eval.head())
# print(train_valid.head())
# print(calendar.head())
# print(sell_price.head())
# print(submission.head())

In [17]:
# read data
print('Reading our data...')
data, submission = read_data()
print(data.shape)
# print(data.head())

Reading our data...
Mem. usage decreased to 1540.89 Mb (57.1% reduction)
(31681090, 18)


In [18]:
#Check if we have enough data available for training
data['date'] = pd.to_datetime(data['date'])
# get amount of unique days in our data
days = abs((data['date'].min() - data['date'].max()).days)
# how many training data do we need to train with at least 2 years and consider lags
need = 365 + 365 + 90 + 28 #90 +28 because we are taking max rolling mean of 90 days and last 28 days are for test data.
print(f'We have {(days - 28)} days of training history')
print(f'we have {(days - 28 - need)} days left')
if (days - 28 - need) > 0:
    print('We have enought training data, lets continue')
else:
    print('Get more training data, training can fail')

We have 1011 days of training history
we have 163 days left
We have enought training data, lets continue


In [19]:
# simple feature engineer
print('Running simple feature engineering...')
data = simple_fe(data)
print(data.shape)
# print(data.head())

Running simple feature engineering...
Mem. usage decreased to 2386.86 Mb (37.8% reduction)
(31681090, 46)


Following is the count and percentage of NAs right after feature engineering.

In [20]:
count_na = pd.concat([pd.DataFrame(data.isnull().sum().reset_index()), pd.DataFrame(np.round((data.isnull().sum().values/data.shape[0])*100, 2))], axis=1)
count_na

,index,0,0
0,id,0,0.00
1,item_id,0,0.00
2,dept_id,0,0.00
3,cat_id,0,0.00
4,store_id,0,0.00
5,state_id,0,0.00
6,demand,0,0.00
7,part,0,0.00
8,date,0,0.00
9,wm_yr_wk,0,0.00


In [21]:
print("Number of unique IDs: ",len(data['id'].unique()))

Number of unique IDs:  30490


A few observation regarding couont of NAs post feature egnineering in above cell are as follows:
* We have total _30490_ unique IDs in the data.
* When we create rolling features for demand variable which doesn't have any NA, we end up having $ 28+ (rolling\_window -1) $ NAs for each of the IDs for each of these rolling features. Since we have 30490 unique IDs, total we will have $ 30490 * (28+ (rolling\_window -1)) $ NAs, where rolling window is an element from $[7, 15, 30, 90]$.
* When we have NAs is _demand_ variable too, then the number of NAs in rolling features of demand will have different count on NAs.
* To get rid of these NAs, we remove first 118 days of data for each of the IDs in the following cell. After this operation we have zero NAs for these rolling _demand_ features.
* With the same logic we can also find the count of NAs for engineered features from _sell_price_ variable.

In [22]:
print('Removing first 118 days')
# eliminate the first 118 days of our train data because of lags/rolling mean (max 90 days) and 28 days for test data.
min_date = data['date'].min() + timedelta(days = 118) #
data = data[data['date'] > min_date]

# NA count after dropping initial 118 days of data
count_na = pd.concat([pd.DataFrame(data.isnull().sum().reset_index()), pd.DataFrame(np.round((data.isnull().sum().values/data.shape[0])*100, 2))], axis=1)
count_na

Removing first 118 days


,index,0,0
0,id,0,0.00
1,item_id,0,0.00
2,dept_id,0,0.00
3,cat_id,0,0.00
4,store_id,0,0.00
5,state_id,0,0.00
6,demand,0,0.00
7,part,0,0.00
8,date,0,0.00
9,wm_yr_wk,0,0.00


In [23]:
min_date= data["date"].min()
max_date= data["date"].max()
print("Start date: ", min_date)
print("End date:", max_date)

Start date:  2013-11-14 00:00:00
End date: 2016-05-22 00:00:00


Following is the number of levels in each of the categorical variables which will go into lightgbm.

In [24]:
cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
           'event_name_2', 'event_type_2']
for col in cat:
    print(col, len(data.loc[:,col].unique()))

item_id 3049
dept_id 7
cat_id 3
store_id 10
state_id 3
event_name_1 31
event_type_1 5
event_name_2 3
event_type_2 3


In [25]:
# define our numeric features and categorical features
features = ["item_id", "dept_id", "cat_id", "store_id", "state_id", "event_name_1", "event_type_1", 
            "event_name_2", "event_type_2", "snap_CA", "snap_TX", "snap_WI", "sell_price",
    # demand features.
    "demand_rolling_mean_t7", "demand_rolling_mean_t15", "demand_rolling_mean_t30", "demand_rolling_mean_t90",
    "demand_rolling_std_t7", "demand_rolling_std_t15", "demand_rolling_std_t30", "demand_rolling_std_t90",
            
    # price features
    "price_change_t1", "price_change_t365", "rolling_price_std_t7", "rolling_price_std_t30",
    # time features.
    "year", "month", "week", "day", "dayofweek", "dayofyear", "is_year_end", "is_year_start", "is_quarter_end", "is_quarter_start",
    "is_month_end", "is_month_start", "is_weekend",
]

# features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'year', 
#                 'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear', 'demand_rolling_mean_t7', 'demand_rolling_mean_t15', 'demand_rolling_mean_t30', 'demand_rolling_mean_t90',
#                 'demand_rolling_std_t7', 'demand_rolling_std_t15', 'demand_rolling_std_t30', 'demand_rolling_std_t90']

cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
                'event_name_2', 'event_type_2']

print('-'*50)
print('\n')
print(f'Training model with {len(features)} features...')
# run lgbm model with 5 GroupKFold (subgroups by year, month)
test = run_lgb(data, features, cat_features)

--------------------------------------------------


Training model with 38 features...
Training fold 1
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.514820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6589
[LightGBM] [Info] Number of data points in the train set: 21800350, number of used features: 38
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[50]	training's custom_asymmetric_eval: 5.05764	valid_1's custom_asymmetric_eval: 5.66431
[100]	training's custom_asymmetric_eval: 4.66307	valid_1's custom_asymmetric_eval: 5.48894
[150]	training's custom_asymmetric_eval: 4.46178	valid_1's custom_asymmetric_eval: 5.42989
[200]	training's custom_asymmetric_eval: 4.31945	valid_1's custom_asymmetric_eval: 5.3

In [26]:
test.head()

,id,date,demand
27227570,HOBBIES_1_001_CA_1_validation,2016-04-25,0.927728
27227571,HOBBIES_1_002_CA_1_validation,2016-04-25,0.314889
27227572,HOBBIES_1_003_CA_1_validation,2016-04-25,0.428381
27227573,HOBBIES_1_004_CA_1_validation,2016-04-25,2.163015
27227574,HOBBIES_1_005_CA_1_validation,2016-04-25,0.986496


In [27]:
print('Save predictions...')
# predict
predict(test, submission)
print("Prediction is generated...")

Save predictions...
Prediction is generated...


In [ ]:
# # helper functions to reduce memory
# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2    
#     for col in df.columns:
#         col_type = df[col].dtypes
#         if col_type in numerics:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)    
#     end_mem = df.memory_usage().sum() / 1024**2
#     if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
#     return df

# # function to read our data
# def read_data():
#     # read data
#     data = pd.read_pickle('/kaggle/input/m5-reduce-data/data_small.pkl')
#     # fillna and label encode categorical features
#     data = transform(data)
#     # read submission
#     submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
#     return data, submission

# # filla na and label encode categorical features
# def transform(data):
    
#     nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
#     for feature in nan_features:
#         data[feature].fillna('unknown', inplace = True)
        
#     cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
#            'event_name_2', 'event_type_2']
#     for feature in cat:
#         encoder = preprocessing.LabelEncoder()
#         data[feature] = encoder.fit_transform(data[feature])
        
#     # reduce memory usage
#     data = reduce_mem_usage(data)
    
#     return data

# # simple feature ingineer function
# def simple_fe(data):
    
#     data_fe = data[['id', 'demand']]
    
#     window = 28
#     periods = [7, 15, 30, 90]
#     group = data_fe.groupby('id')['demand']
    
#     # most recent lag data
#     for period in periods:
#         data_fe['demand_rolling_mean_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).mean())
#         data_fe['demand_rolling_std_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).std())
        
#     # reduce memory
#     data_fe = reduce_mem_usage(data_fe)
    
#     # get time features
#     data['date'] = pd.to_datetime(data['date'])
#     time_features = ['year', 'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear']
#     dtype = np.int16
#     for time_feature in time_features:
#         data[time_feature] = getattr(data['date'].dt, time_feature).astype(dtype)
        
#     # concat lag and rolling features with main table
#     lag_rolling_features = [col for col in data_fe.columns if col not in ['id', 'demand']]
#     data = pd.concat([data, data_fe[lag_rolling_features]], axis = 1)
    
#     del data_fe
#     gc.collect()

#     return data

# # define custom loss function
# def custom_asymmetric_train(y_pred, y_true):
#     y_true = y_true.get_label()
#     residual = (y_true - y_pred).astype("float")
#     grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
#     hess = np.where(residual < 0, 2, 2 * 1.15)
#     return grad, hess

# # define custom evaluation metric
# def custom_asymmetric_valid(y_pred, y_true):
#     y_true = y_true.get_label()
#     residual = (y_true - y_pred).astype("float")
#     loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * 1.15) 
#     return "custom_asymmetric_eval", np.mean(loss), False

# # define lgbm simple model
# def run_lgb(data, features, cat_features):
    
#     # reset_index
#     data.reset_index(inplace = True, drop = True)
    
#     # going to evaluate with the last 28 days
#     x_train = data[data['date'] <= '2016-04-24']
#     y_train = x_train['demand']
#     test = data[data['date'] >= '2016-04-25']

#     # define random hyperparammeters
#     params = {
#         'boosting_type': 'gbdt',
#         'n_jobs': -1,
#         'seed': 42,
#         'learning_rate': 0.1,
#         'bagging_fraction': 0.85,
#         'bagging_freq': 1, 
#         'colsample_bytree': 0.85,
#         'colsample_bynode': 0.85,
#         'min_data_per_leaf': 25,
#         'num_leaves': 200,
#         'lambda_l1': 0.5,
#         'lambda_l2': 0.5}
    
#     oof = np.zeros(len(x_train))
#     preds = np.zeros(len(test))
    
#     # GroupKFold by week, month to avoid leakage and overfitting (not entirely sure xD)
#     kf = GroupKFold(5)
#     # get subgroups for each week, year pair
#     group = x_train['week'].astype(str) + '_' + x_train['year'].astype(str)
#     for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train, group)):
#         print(f'Training fold {fold + 1}')
#         train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], 
#                                 categorical_feature = cat_features)
#         val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], 
#                               categorical_feature = cat_features)
        
#         # train with our custom loss function and evaluation metric
#         model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
#                           valid_sets = [train_set, val_set], verbose_eval = 50, fobj = custom_asymmetric_train, 
#                           feval = custom_asymmetric_valid)
    
#         # predict oof
#         oof[val_idx] = model.predict(x_train.iloc[val_idx][features])

#         # predict test
#         preds += model.predict(test[features]) / 5
        
#         print('-'*50)
#         print('\n')
        
#     oof_rmse = np.sqrt(metrics.mean_squared_error(y_train, oof))
#     print(f'Our out of folds rmse is {oof_rmse}')
        
#     test = test[['id', 'date', 'demand']]
#     test['demand'] = preds
#     return test

# # function to get the predictions in the correct format
# def predict(test, submission):
#     predictions = pd.pivot(test, index = 'id', columns = 'date', values = 'demand').reset_index()
#     predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

#     evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
#     evaluation = submission[submission['id'].isin(evaluation_rows)]

#     validation = submission[['id']].merge(predictions, on = 'id')
#     final = pd.concat([validation, evaluation])
#     final.to_csv('submission_custom_loss.csv', index = False)
    
# # this is the main function that will run our entire program
# def train_and_evaluate():
    
#     # read data
#     print('Reading our data...')
#     data, submission = read_data()
    
#     data['date'] = pd.to_datetime(data['date'])
#     # get amount of unique days in our data
#     days = abs((data['date'].min() - data['date'].max()).days)
#     # how many training data do we need to train with at least 2 years and consider lags
#     need = 365 + 365 + 90 + 28
#     print(f'We have {(days - 28)} days of training history')
#     print(f'we have {(days - 28 - need)} days left')
#     if (days - 28 - need) > 0:
#         print('We have enought training data, lets continue')
#     else:
#         print('Get more training data, training can fail')
    
#     # simple feature engineer
#     print('Running simple feature engineering...')
#     data = simple_fe(data)
#     print('Removing first 118 days')
#     # eliminate the first 118 days of our train data because of lags
#     min_date = data['date'].min() + timedelta(days = 118)
#     data = data[data['date'] > min_date]
    
#     # define our numeric features and categorical features
#     features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'year', 
#                 'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear', 'demand_rolling_mean_t7', 'demand_rolling_mean_t15', 'demand_rolling_mean_t30', 'demand_rolling_mean_t90',
#                 'demand_rolling_std_t7', 'demand_rolling_std_t15', 'demand_rolling_std_t30', 'demand_rolling_std_t90']
    
#     cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
#                     'event_name_2', 'event_type_2']
    
#     print('-'*50)
#     print('\n')
#     print(f'Training model with {len(features)} features...')
#     # run lgbm model with 5 GroupKFold (subgroups by year, month)
#     test = run_lgb(data, features, cat_features)
#     print('Save predictions...')
#     # predict
#     predict(test, submission)
        
# # run our program
# train_and_evaluate()
